## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-17-22-36-22 +0000,nyt,Zelensky Brings Backup to the White House as T...,https://www.nytimes.com/2025/08/17/us/politics...
1,2025-08-17-22-36-17 +0000,nypost,"US tourist, 63, attacked by shark, severely in...",https://nypost.com/2025/08/17/world-news/us-to...
2,2025-08-17-22-30-35 +0000,nyt,"Ronnie Rondell, Stuntman Set on Fire for Pink ...",https://www.nytimes.com/2025/08/17/us/ronnie-r...
3,2025-08-17-22-29-56 +0000,nypost,More Palestinians are calling for Hamas to lea...,https://nypost.com/2025/08/17/world-news/more-...
4,2025-08-17-22-29-37 +0000,nyt,President’s Advisers Tout Progress in Summit W...,https://www.nytimes.com/live/2025/08/17/us/tru...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2309/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
5,trump,26
8,putin,16
162,ukraine,14
0,zelensky,8
147,war,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
150,2025-08-17-01-10-58 +0000,nypost,First Lady Melania Trump calls for end to war ...,https://nypost.com/2025/08/16/world-news/first...,89
136,2025-08-17-06-42-55 +0000,bbc,"Russia complicating end to war, says Zelensky,...",https://www.bbc.com/news/articles/c04rv2p3936o...,69
101,2025-08-17-12-18-22 +0000,nypost,"European leaders, NATO boss will back up Zelen...",https://nypost.com/2025/08/17/world-news/europ...,67
65,2025-08-17-17-51-38 +0000,nypost,Ukraine won’t give up land that Russia doesn’t...,https://nypost.com/2025/08/17/world-news/ukrai...,65
64,2025-08-17-17-55-55 +0000,nypost,Pence urges Trump to take out the ‘hammer’ on ...,https://nypost.com/2025/08/17/us-news/mike-pen...,64


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
150,89,2025-08-17-01-10-58 +0000,nypost,First Lady Melania Trump calls for end to war ...,https://nypost.com/2025/08/16/world-news/first...
101,41,2025-08-17-12-18-22 +0000,nypost,"European leaders, NATO boss will back up Zelen...",https://nypost.com/2025/08/17/world-news/europ...
63,36,2025-08-17-18-12-56 +0000,bbc,Three Republican-led states to send National G...,https://www.bbc.com/news/articles/cvgv1pmypdyo...
58,31,2025-08-17-18-52-45 +0000,nypost,Nationwide strike erupts across Israel demandi...,https://nypost.com/2025/08/17/world-news/natio...
37,30,2025-08-17-20-01-23 +0000,nypost,NYC hookah lounge where 12 were shot Sunday wa...,https://nypost.com/2025/08/17/us-news/nyc-hook...
10,24,2025-08-17-21-57-52 +0000,nypost,Air Canada delays restart after flight attenda...,https://nypost.com/2025/08/17/business/air-can...
114,24,2025-08-17-10-00-00 +0000,latimes,News Analysis: Newsom's decision to fight fire...,https://www.latimes.com/california/story/2025-...
89,24,2025-08-17-14-37-47 +0000,nypost,HUD launches English-only initiative for all d...,https://nypost.com/2025/08/17/us-news/hud-laun...
11,23,2025-08-17-21-45-24 +0000,nypost,New Yorkers fighting against massive battery s...,https://nypost.com/2025/08/17/us-news/new-york...
44,23,2025-08-17-19-36-42 +0000,nypost,Lindsey Graham threatens bill designating Russ...,https://nypost.com/2025/08/17/us-news/lindsey-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
